# Бинарный поиск

Бинпоиск (или двоичный) бывает трех видов:
- целочисленный
- вещественный
- по ответу

## Целочисленный бинпоиск

Имеется: массив целых чисел `a`, отсортированный по неубыванию, имеющий размер $n\approx10^6$

Мы хотим научится отвечать на запросы вида "найти первое вхождение элемента $k$", количество запросов $q\approx 10^5$

Если мы будем отвечать на эти запросы линейным поиском, то в общей сложности получим $10^5\cdot 10^6=10^{11}$, что очень долго

Давайте создадим *фиктивные элементы* в начале и в конце массива и поставим на них два указателя: `l` - указывает на *элемент* в начале, и `r` - на *элемент* в конце. Тогда, мы сможем вычислить `mid` - указатель на середину массива. 

Так как массив отсортирован по неубыванию, то сравнив `a[mid]` и `k` мы сможем однозначно понять где лежит элемент `k` — в `a[l:mid]` или `a[mid:r]`. Поняв это, мы "отбрасываем" ту часть, где нет `k`, переставляя указатель
- если `k` оказался в `a[:mid]`, то мы двигаем указатель `r`
- если `k` в `a[mid:]`, то двигаем `l`

**Пример.** Пусть `a = [2, 2, 4, 4, 4, 5, 7, 7, 9]` и `k=6`. Создаем указатели: `l=-1, r=len(a)=9`. 

<img src="img/binsearch1-alt.jpg">

- Вычисляем `mid = (l + r) // 2 = 4`

<img src="img/binsearch2-alt.jpg">

- Сравниваем: `a[mid]=4<6=k`, следовательно `k` находится в правой части, значит сдвигаем `l` на `mid`
- Снова вычисляем `mid = (4+8)//2 = 6`

<img src="img/binsearch3-alt.jpg">

- Сравниваем: `a[mid]=7>6=k`, следовательно `k` находится в левой части отрезка `[l:r]`, значит сдвигаем `r` на `mid`
- Снова вычисляем `mid = (4+6)//2 = 5`

<img src="img/binsearch4-alt.jpg">

- Сравниваем: `a[mid]=5<6=k`, следовательно `k` находится в правой части отрезка `[l:r]`, значит сдвигаем `l` на `mid`
- Получаем, что `l=5, r=6` и мы понимаем, что элемента `k` просто нет. `l` и `r` находятся рядом, дальше нет возможности вычислить `mid` 

Если бы элементов было четное количество, то `mid` вычислялся бы так $$\left\lfloor\frac{l+r}{2}\right\rfloor$$

In [1]:
a = [2, 2, 4, 4, 4, 5, 7, 7, 9]
b = [2, 2, 2, 4, 4, 4, 5, 6, 9, 9]

def bin_first(a, k):    # первое вхождение
    l, r = -1, len(a)
    while l + 1 < r:
        m = (l + r) // 2
        if a[m] < k:
            l = m
        else:
            r = m

    if r != len(a) and a[r] == k:
        return r
    else:
        return 'нет такого элемента'


print(bin_first(a, 6))
print(bin_first(b, 4))

нет такого элемента
3


In [3]:
def bin_last(a, k):     # последнее вхождение
    l, r = -1, len(a)
    while l + 1 < r:
        m = (l + r) // 2
        if a[m] <= k:
            l = m
        else:
            r = m

    if l != -1 and a[l] == k:
        return l
    else:
        return 'нет такого элемента'

print(bin_last(a, 7))    
print(bin_last(b, 4))
print(bin_last([], 4))

7
5
нет такого элемента


## Вещественный бинпоиск

Пусть имеется функция $f(x)$, определенная на отрезке $[a;b]$ и мы можем вычислить значение функции на любой точке отрезка.

Мы хотим вычислить при каком $x$ функция $f(x)=0$ на отрезке $[a;b]$. И делаем это бинпоиском:
- вычисляем `m=(a+b)/2`
- если знаки `f(a)` и `f(m)` отличаются, значит ноль этой функции находится в $(a;m)\Longrightarrow$ двигаем правую границу - $b$
- но если знаки `f(a)` и `f(m)` одинаковы, то ноль функции лежит в $(m;b)$, то есть двигаем левую границу $a$
- двигаем границы до тех пор, пока не сузим отрезок $[a;b]$ до малой окрестности, например, $10^{-6}$

<img src='img/binsearch5.jpg'>

**Примечание.** Бинпоиск в этом случае работает, только если на отрезке $[a;b]$ единственный корень. Если на отрезке 2 и более корней, то этот отрезок нужно *разбить* на несколько более мелких.

In [13]:
def f(x:float) -> float:
    return x * x - 3
    
a, b = 0, 3
while b - a > 0.0000001:
    m = (a + b) / 2
    if f(a) * f(m) > 0:
        a = m
    else:
        b = m

x = (a + b) / 2
print(x, m, sep='\n')    # в качестве ответа можно взять как x, так и m 

1.732050821185112
1.7320508658885956


## Бинпоиск по ответу

Бинарный поиск по ответу применяется, когда у нас нет массива, но есть функция-чекер `ok(x)`, которая для любого числа $x$ может определить, подходит ли это значение как потенциальный ответ. 

Главное, что результаты вызова функции-чекера должны быть монотонными — если `ok(x)` стала равной `True`, то для всех больших $x$ результат не меняется. И наоборот - если она стала равной `False`, то для всех больших $x$ результат не изменится.

В таком случае бинарный поиск позволяет найти то значение $x$, в котором происходит переход: самое маленькое $x$, для которого `ok(x)=True`, или наоборот — самое большое `x`, для которого `ok(x)=True`.

**Задача. Принтеры.** Есть два принтера, первый печатает 1 лист за 3 сек., а второй - за 5 сек. Мы одновременно запускаем на них печать $7$ листов. Вопрос: через сколько времени мы получим все 7 листов. 

Мы могли бы создать массив `a`, где индекс обозначает время, а по индексу хранится количество страниц, которое мы сможем напечатать за соответствующее время. Тогда, массив выглядел бы так
$$
[0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 5, 5, 6, 6, 6, 8]
$$
Проблема в том, что это будет затратно по времени и памяти. Давайте тогда не будем создавать его, а используем бинпоиск.

In [19]:
l, r = 0, 100
k = 7
while l + 1 < r:
    m = (l + r) // 2
    cnt = m // 3 + m // 5
    if cnt < k:
        l = m
    else:
        r = m

print(r)

15


Фактически, мы не создаем массив, а в моменте считаем конкретные значения его ячеек, то есть `a[m]=cnt`